In [2]:
import pandas as pd
import numpy as np
utility_matrix = pd.read_csv("50%_utility_matrix.csv")
feature_matrix = pd.read_csv("feature_matrix.csv")

In [14]:
#create dictionary to map anime_id to its genre vector
feature_matrix_indexed = feature_matrix.set_index('anime_id')
anime_vector = feature_matrix_indexed['genre_vector'].to_dict()
for key in anime_vector.keys():
    anime_vector[key] = eval(anime_vector[key])
print(anime_vector)

{1: [1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 5: [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 6: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 7: [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 8: [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 15: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 16: [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 17: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [58]:
def getRatingVectorOfNthUser(n:int):
    #get the rating vector of n-th user 
    column_n = str(n)
    rating = utility_matrix[[column_n]].values
    animeId_withMinus1Rating = utility_matrix[utility_matrix[column_n] == -1][['anime_id']].values
    minus1_vector = []
    for i in range(len(animeId_withMinus1Rating)):
        minus1_vector.append(anime_vector[animeId_withMinus1Rating[i][0]])
    minus1_vector = np.array(minus1_vector)
    n1,_ = minus1_vector.shape
    #normalize the rating value
    count_matrix = (1/n1)*sum(minus1_vector)
    return rating,count_matrix.dot(minus1_vector[4])*10 #rating too small

print(getRatingVectorOfNthUser(1))

(array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]]), 28.053691275167782)


In [4]:
def similarity(u1:int,u2:int) -> float:
    vector_u1 = getRatingVectorOfNthUser(u1)
    vector_u2 = getRatingVectorOfNthUser(u2)
    
    return ((vector_u1.T).dot(vector_u2))/((np.linalg.norm(vector_u1))*np.linalg.norm(vector_u2))

print(similarity(1,10)[0][0])

0.216788524485329


In [6]:
#calculate the similarity matrix of all users in our dataset
#this is not necessary for the prediction but  can be useful to cluster the users into distinct groups
similar_matrix = [[0 for i in range(1000)] for j in range(1000)]
print(np.array(similar_matrix))
for i in range(len(similar_matrix)):
    for j in range(len(similar_matrix)):
        if i == j:
            similar_matrix[i][j] = 1
        if similar_matrix[j][i] != 0:
            similar_matrix[i][j] = similar_matrix[j][i]
        else:
            similar_matrix[i][j] = similarity(i+1,j+1)[0][0]
print(np.array(similar_matrix))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[ 1.         -0.04225847  0.01613407 ... -0.03050808  0.01968847
   0.11480223]
 [-0.04225847  1.          0.13368172 ...  0.          0.
   0.        ]
 [ 0.01613407  0.13368172  1.         ...  0.05790612  0.2762701
   0.1151693 ]
 ...
 [-0.03050808  0.          0.05790612 ...  1.          0.09421744
   0.06584984]
 [ 0.01968847  0.          0.2762701  ...  0.09421744  1.
   0.13113166]
 [ 0.11480223  0.          0.1151693  ...  0.06584984  0.13113166
   1.        ]]


In [5]:
user = 1
max_similarity = 0
list_of_max = []
for i in range(2,36758):
    if similarity(user,i)[0][0] > max_similarity:
        max_similarity = similarity(user,i)[0][0]
        list_of_max.append((i,max_similarity))
print(list_of_max[:-4])

[(3, 0.016134070766122124), (8, 0.04511555413480946), (10, 0.216788524485329)]
